<a href="https://colab.research.google.com/github/gustavo-moretto/WebScrapping_ImoveisCuritiba/blob/main/imoveis_curitiba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
comprar_alugar = ['venda', 'aluguel'] 

In [37]:
def analise_imoveis( n_paginas, 
                    comprar_alugar=['venda', 'aluguel'], 
                    tipos_imoveis = ['apartamento_residencial', 'casa_residencial', 'condominio_residencial', 'cobertura_residencial', 'flat_residencial', 'kitnet_residencial', 'sobrado_residencial']):

  import requests
  import pandas as pd
  from bs4 import BeautifulSoup

  import seaborn as sns
  import matplotlib.pyplot as plt 

  conjunto_imoveis = {}
  c = 0
  for negocio in comprar_alugar:
    for pagina in range( n_paginas ):
        for tipo_imovel in tipos_imoveis:  
          # faz a requisição do site e verifica se esta foi bem-sucedida
          req = requests.get(f"https://www.vivareal.com.br/{negocio}/parana/curitiba/{tipo_imovel}/?pagina={pagina}")

          if req.status_code == 200:
              #print(f'Pag. {pagina}: Requisição bem sucedida!')
              content = req.content
          else:
            print(req.status.code)
            break
            
          # lê o arquivo html com a biblioteca BeautifulSoup
          soup = BeautifulSoup(content, 'html.parser')
            
          # seleciona os dados dos imóveis
          # para isso, encontramos todas as tags <a> com a "class" = "property-card__content-link js-card-title"
          imoveis = soup.find_all('a', attrs={'class':"property-card__content-link js-card-title"})
            
          for imovel in range(len(imoveis)):        
              c += 1
                
              indice_imovel = c
              # extrai a descrição do imóvel
              #descricao = imoveis[imovel].find("span", attrs={"class":"property-card__title js-cardLink js-card-title"}).text                 

              # extrai o endereço e o bairro do imóvel
              endereco = imoveis[imovel].find("span", attrs={"class":"property-card__address"}).text
              end_completo = endereco

              for elemento in range(len(endereco.split())):
                  if endereco.split()[elemento] == '-':
                      end = endereco.split()[elemento + 1]                  

                      # bairros com nomes compostos -> dessa forma, o nome eh extraido da maneira correta                 
                      if end in ['Santa', 'Sitio', 'Cristo', 'Vista', 'Boa', 'Água', 'Cidade', 'Campo', 'Capão']:
                        endereco = endereco.split()[elemento + 1] + ' ' + endereco.split()[elemento + 2]
                        # excluir a virgula do final do bairro
                        endereco = endereco[:-1]                    
                      elif end == 'Alto':
                        endereco = endereco.split()[elemento + 1] + ' ' + endereco.split()[elemento + 2] + ' ' + endereco.split()[elemento + 3]
                        # excluir a virgula do final do bairro
                        endereco = endereco[:-1]
                      
                      # caso nao haja o nome da rua, o bairro sera igual a 'PR'
                      # para extrair o bairro, devemos extrair o primeiro elemento da lista
                      elif end == 'PR':
                        endereco = endereco.split()[0][:-1]
                      else:
                        endereco = endereco.split()[elemento + 1]
                        # excluir a virgula do final do bairro
                        endereco = endereco[:-1]
                      break
              # extrai a área do imóvel (m²)
              # como existem imoveis com a area variavel, p. ex. 20-30, criei uma excecao
              # neste caso, a area do imovel sera igual a 0
              # creio que isso traz mais realidade ao dataset tambem
              try:
                area_m2 = float(imoveis[imovel].find("span", attrs={"class":"property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area"}).text)
              except ValueError:
                area_m2 = 0

              # extrai os itens do imóvel, como mobília, churrasqueira, academia...
              #try:
                  #itens = imoveis[imovel].find("ul", attrs={"class":"property-card__amenities"}).text.split()
              #except AttributeError:
                  #itens = []
              #try:
                  #if itens[-1] == '...':
                      #itens = itens[:-1]
                #except IndexError:
                    #pass

              # número de comodos (caso não possua vaga de garagem, por exemplo, será atribuído valor 0)
              comodos = []
              try:
                  for k in imoveis[imovel].find_all("span", attrs={"class":"property-card__detail-value js-property-card-value"}):
                      comodos.append(float(k.text))
              except ValueError:
                  comodos.append(0)
              if len(comodos) == 3:
                quartos = float(comodos[0])
                banheiros = float(comodos[1])
                vagas = float(comodos[2])
              else:
                quartos = 0
                banheiros = 0
                vagas = 0

              # extrai o valor do aluguel
              try: 
                preco = imoveis[imovel].find("p", attrs={'style':"display: block;"}).text                                                         
                try:          
                  if (negocio == 'aluguel' and len(preco.split()[1]) >= 5):            
                    preco = float(preco.split()[1])      
                    preco = preco * 1000
                  elif (negocio == 'venda' and len(preco.split()[1]) >= 5)
                  else:
                    preco = float(preco.split()[1])                                  
                except ValueError:
                  print(negocio, tipo_imovel, preco.split())            
                  preco = 0
              except AttributeError:
                print(negocio, tipo_imovel, preco)
                preco = 0                     

              # extrai o valor do condominio (se não existir, será 0)
              # no laco, caso o valor do condominio seja superior a 999.99 reais, sera multiplicado por 1000
              try:
                  condominio = imoveis[imovel].find("strong", attrs={'class':"js-condo-price"}).text
                  if len(condominio.split()[1]) == 3:
                    condominio = float(condominio.split()[1])
                  else:
                    condominio = float(condominio.split()[1])*1000             
              except AttributeError or ValueError:
                  condominio = 0

              conjunto_imoveis[c] = (negocio, tipo_imovel, indice_imovel, end_completo, endereco, area_m2, quartos, banheiros, vagas, preco, condominio)

    df = pd.DataFrame.from_dict(conjunto_imoveis)
    df = df.T
    df = df.rename(columns={ 0:'comprar_alugar', 1:'tipo_imovel', 2:'indice', 3:'end_completo', 4:'bairro', 5:'area_m2', 6:'quartos', 7:'banheiros', 8:'vagas_garagem', 9:'preco', 10:'condominio'})

    colunas_float = ['area_m2', 'quartos', 'banheiros', 'vagas_garagem', 'preco', 'condominio']
    for coluna in colunas_float: 
      df[coluna] = df[coluna].astype( float )
      
    print(f'Base de dados criada: {len(df)} imoveis cadastrados.')
    return df

In [38]:
df = analise_imoveis( 1 )
df.head()

venda apartamento_residencial ['A', 'partir', 'de', 'R$', '171.000']
venda apartamento_residencial ['R$', '3.799.000']
venda apartamento_residencial ['R$', '1.699.000']
venda apartamento_residencial ['R$', '1.719.000']
venda casa_residencial ['R$', '2.250.000', 'Preço', 'abaixo', 'do', 'mercado']
venda casa_residencial ['R$', '4.180.000']
venda casa_residencial ['R$', '1.790.000']
venda casa_residencial ['R$', '2.990.000']
venda casa_residencial ['R$', '1.300.000']
venda casa_residencial ['R$', '8.200.000']
venda casa_residencial ['R$', '1.800.000']
venda casa_residencial ['R$', '1.000.000', 'Preço', 'abaixo', 'do', 'mercado']
venda casa_residencial ['R$', '1.790.000']
venda casa_residencial ['R$', '1.600.000']
venda casa_residencial ['R$', '1.170.000', 'Preço', 'abaixo', 'do', 'mercado']
venda casa_residencial ['R$', '1.900.000']
venda casa_residencial ['R$', '1.080.000']
venda casa_residencial ['R$', '2.950.000']
venda casa_residencial ['R$', '1.080.000']
venda casa_residencial ['R$'

,comprar_alugar,tipo_imovel,indice,end_completo,bairro,area_m2,quartos,banheiros,vagas_garagem,preco,condominio
1,venda,apartamento_residencial,1,"Avenida Visconde de Guarapuava, 2764 - Centro,...",Centro,69.0,2.0,2.0,1.0,590000.0,650.0
2,venda,apartamento_residencial,2,"Avenida Visconde de Guarapuava, 3070 - Centro,...",Centro,0.0,1.0,1.0,0.0,0.0,0.0
3,venda,apartamento_residencial,3,"Rua Carlos Dietzsch, 475 - Portão, Curitiba - PR",Portão,33.0,1.0,1.0,1.0,225000.0,350.0
4,venda,apartamento_residencial,4,"Avenida Iguaçu, 2663 - Batel, Curitiba - PR",Batel,220.0,4.0,5.0,3.0,0.0,2800.0
5,venda,apartamento_residencial,5,"Rua Coronel Pedro Scherer Sobrinho, 152 - Cris...",Cristo Rei,182.0,3.0,5.0,3.0,0.0,1000.0


In [29]:
df[df['preco'] == 0]

,comprar_alugar,tipo_imovel,indice,end_completo,bairro,area_m2,quartos,banheiros,vagas_garagem,preco,condominio
2,venda,apartamento_residencial,2,"Avenida Visconde de Guarapuava, 3070 - Centro,...",Centro,0.0,1.0,1.0,0.0,0.0,0.0
4,venda,apartamento_residencial,4,"Avenida Iguaçu, 2663 - Batel, Curitiba - PR",Batel,220.0,4.0,5.0,3.0,0.0,2800.0
5,venda,apartamento_residencial,5,"Rua Coronel Pedro Scherer Sobrinho, 152 - Cris...",Cristo Rei,182.0,3.0,5.0,3.0,0.0,1000.0
13,venda,apartamento_residencial,13,"Avenida Anita Garibaldi, 491 - Ahú, Curitiba - PR",Ahú,213.0,4.0,5.0,2.0,0.0,1400.0
40,venda,casa_residencial,40,"Rua Maranhão, 349 - Água Verde, Curitiba - PR",Água Verde,700.0,4.0,5.0,6.0,0.0,345.0
...,...,...,...,...,...,...,...,...,...,...,...
232,venda,sobrado_residencial,232,"Rua Doutor Bley Zornig, 3865 - Boqueirão, Curi...",Boqueirão,0.0,3.0,2.0,1.0,0.0,0.0
237,venda,sobrado_residencial,237,"Rua Carlos Essenfelder, 4507 - Boqueirão, Curi...",Boqueirão,224.0,3.0,3.0,15.0,0.0,0.0
244,venda,sobrado_residencial,244,"Pilarzinho, Curitiba - PR",Pilarzinho,300.0,4.0,2.0,5.0,0.0,0.0
245,venda,sobrado_residencial,245,Rua Lysimaco Ferreira da Costa - Centro Cívico...,Centr,300.0,3.0,2.0,3.0,0.0,0.0


In [30]:
len(df)

252

# Imports

In [5]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# Analise Descritiva dos Dados

In [3]:
# analise descritiva dos dados
df.describe( include='all' )

,indice,end_completo,bairro,area_m2,quartos,banheiros,vagas_garagem,aluguel,condominio
count,7200.0,7200,7200,7200.000000,7200.000000,7200.00000,7200.000000,7200.000000,7200.000000
unique,7200.0,35,19,NaN,NaN,NaN,NaN,NaN,NaN
top,1.0,"Alameda Júlia da Costa, 911 - Bigorrilho, Curi...",Bigorrilho,NaN,NaN,NaN,NaN,NaN,NaN
freq,1.0,400,1600,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,82.888889,2.277778,1.75000,1.194444,2534.666667,535.916667
std,NaN,NaN,NaN,50.183039,0.960712,0.95386,0.700027,1415.087849,440.586961
min,NaN,NaN,NaN,25.000000,1.000000,1.00000,0.000000,950.000000,0.000000
25%,NaN,NaN,NaN,43.750000,1.750000,1.00000,1.000000,1475.000000,350.000000
50%,NaN,NaN,NaN,66.500000,2.000000,1.00000,1.000000,2000.000000,418.000000
75%,NaN,NaN,NaN,111.500000,3.000000,2.00000,2.000000,3150.000000,668.750000


In [6]:
# verificando o nome dos bairros
bairros = pd.Series.to_list(df['bairro'].drop_duplicates())

# na coleta dos dados, um bairro acaba ficando como PR ou P
# vamos verificar quantos registros possuem este criterio como bairro
print(f"Numero de registros com bairro igual a PR: {len(df[(df['bairro'] == 'PR') | (df['bairro'] == 'P')])}")

Numero de registros com bairro igual a PR: 0


## Apartamentos por Bairro

In [7]:
df[['bairro' , 'indice']].groupby('bairro').count()

,indice
bairro,
Alto da Glória,200
Bacacheri,200
Barreirinha,200
Batel,800
Bigorrilho,1600
Cabral,200
Cajuru,400
Camp,200
Centro,800
